# The formal version of SalesOrder.ipynb

####  Simulate all input data

In [6]:
import random
from datetime import datetime, timedelta
import pandas as pd
import sqlite3

# Step 1: Generate Orders and Sales Data

# Sales and customer data
sales_ids = ['Sales A', 'Sales B', 'Sales C', 'Sales D', 'Sales E']
sales_names = ['Alice', 'Bob', 'Charlie', 'David', 'Eve']
customer_ids = [f'Customer {i}' for i in range(1, 21)]
order_ids = [f'2024-{str(i).zfill(2)}' for i in range(1, 51)]

# Function to get the start date of a specific week in a given year
def get_week_start(year, week):
    return datetime.strptime(f'{year}-W{str(week).zfill(2)}-1', "%Y-W%W-%w")

# Initialize orders list
orders = []

# Generate random order dates, delivery dates, and factories
for order_id in order_ids:
    order_week = int(order_id.split('-')[1])
    order_week_start = get_week_start(2024, order_week)
    
    # Generate random order and delivery dates
    order_date = order_week_start - timedelta(weeks=random.choice([1, 2]), days=random.randint(0, 6))
    delivery_date = order_week_start + timedelta(weeks=random.choice([4, 5]), days=random.randint(0, 6))
    factory = random.choice(['China', 'Vietnam'])
    
    orders.append({
        'Order ID': order_id,
        'Order Date': order_date.strftime('%Y-%m-%d'),
        'Delivery Date': delivery_date.strftime('%Y-%m-%d'),
        'Factory': factory
    })

# Step 2: Generate Sales Orders Data
data = []

# Assign each sales rep to approximately 5 customers
for sales_id, sales_name in zip(sales_ids, sales_names):
    assigned_customers = random.sample(customer_ids, k=5)
    for customer in assigned_customers:
        # Each customer will have 3 to 4 orders
        num_orders = random.randint(3, 4)
        for _ in range(num_orders):
            order_id = random.choice(order_ids)
            yield_rate = round(random.uniform(96.1, 99.8), 2)
            thru_put = round(random.uniform(950, 1200), 2)
            
            # Find the matching order details from the orders list
            order_details = next((order for order in orders if order['Order ID'] == order_id), None)
            
            if order_details:
                # Append the combined data to the data list
                data.append([
                    sales_id, 
                    sales_name, 
                    customer,  # Correctly refer to the single customer
                    order_id, 
                    yield_rate, 
                    thru_put, 
                    order_details['Order Date'], 
                    order_details['Delivery Date'], 
                    order_details['Factory']
                ])

# Step 3: Convert the Data to a DataFrame
columns = ['sales_id', 'sales_name', 'customer_id', 'order_id', 'yield_rate', 
           'thru_put', 'order_date', 'delivery_date', 'factory']
df = pd.DataFrame(data, columns=columns)

# Save the DataFrame to a CSV file
csv_filename = 'sales_orders.csv'  #----------- .csv---------------------
df.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' has been created successfully.")

# Step 4: Import CSV into SQLite Database
db_filename = 'sales_orders.db'  #------------.db --------------------
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()

# Create a new table in the SQLite database
table_name = 'sales_orders' # ----------------------database table----------
cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

# Define the table schema with the additional fields
create_table_query = f'''
CREATE TABLE {table_name} (
    "sales_id" TEXT,
    "sales_name" TEXT,
    "customer_id" TEXT,
    "order_id" TEXT,
    "yield_rate" REAL,
    "thru_put" REAL,
    "order_date" TEXT,
    "delivery_date" TEXT,
    "factory" TEXT
)
'''
cursor.execute(create_table_query)

# Insert the data from the DataFrame into the SQLite database
df.to_sql(table_name, conn, if_exists='append', index=False)

# Commit and close the database connection
conn.commit()
conn.close()

print(f"Data from '{csv_filename}' has been successfully imported into '{db_filename}' database, table '{table_name}'.")

# -------------- confirmation of simulated input data is complete----------
# can click sales_orders.csv and use DBeaver to view the data
# can use the following code to load the data from the database
df = pd.read_sql(f"SELECT * FROM {table_name}", sqlite3.connect(db_filename))
print(df.head())  # Display the first few rows of the DataFrame


CSV file 'sales_orders.csv' has been created successfully.
Data from 'sales_orders.csv' has been successfully imported into 'sales_orders.db' database, table 'sales_orders'.
  sales_id sales_name  customer_id order_id  yield_rate  thru_put  order_date  \
0  Sales A      Alice   Customer 9  2024-25       96.48   1044.94  2024-06-09   
1  Sales A      Alice   Customer 9  2024-21       99.33   1155.38  2024-05-11   
2  Sales A      Alice   Customer 9  2024-23       99.76   1076.72  2024-05-26   
3  Sales A      Alice  Customer 15  2024-32       97.65    978.33  2024-07-23   
4  Sales A      Alice  Customer 15  2024-11       98.95   1065.96  2024-03-04   

  delivery_date  factory  
0    2024-07-25    China  
1    2024-06-28  Vietnam  
2    2024-07-12    China  
3    2024-09-08  Vietnam  
4    2024-04-13    China  


In [ ]:
from ast import Name
import random
from datetime import datetime, timedelta
import pandas as pd
import sqlite3

# Define the function
def get_customers_by_sales_name(sales_name: str) -> list[str]:
    """
    Fetch distinct customer IDs based on the given sales name.
    """
    conn = sqlite3.connect("sales_orders1.db")
    with conn:
        cursor = conn.cursor()
        cursor.execute("SELECT DISTINCT customer FROM sales_orders1 WHERE sales_name = ?", (sales_name,))
        customers = [row[0] for row in cursor.fetchall()]
    # Debug print to confirm correct return value
    print(f"Debug: Retrieved customers for sales_name '{sales_name}': {customers}")
    return customers

# Define sales_name with a valid value
sales_name = "Alice"  # Replace with a valid sales_name in your database

# Test the function
customers = get_customers_by_sales_name(sales_name)
print("Final returned customers:", customers)


## for solving one customer corresponding to many sales

In [7]:
import random
from datetime import datetime, timedelta
import pandas as pd
import sqlite3

def generate_sales_data(csv_filename='sales_orders.csv',db_filename='sales_orders.db', table_name='sales_orders'):
    # Updated sales IDs and names
    sales_ids = ['Sales_A', 'Sales_B', 'Sales_C', 'Sales_D', 'Sales_E']
    sales_names = ['Alice', 'Bob', 'Charlie', 'David', 'Eve']
    print("Sales IDs and Names:")
    for sid, sname in zip(sales_ids, sales_names):
        print(f"{sid}: {sname}")
    print()  # Blank line for clarity

    # Updated customer names from the cooling fan industry
    customer_ids = [
        'Foxconn', 'Delta_Electronics', 'Nidec', 'SanAce', 'Cooler_Master',
        'Be_Quiet', 'Noctua', 'Corsair', 'NZXT', 'Arctic',
        'Phanteks', 'Thermaltake', 'Fractal_Design', 'Lian_Li', 'DeepCool',
        'AeroCool', 'SilverStone', 'Cougar', 'Scythe', 'Antec'
    ]
    print("Customer IDs:")
    print(customer_ids)
    print()

    # Assign each sales representative a random number of customers (2 to 6)
    assigned_customers = {}
    available_customers = customer_ids[:]
    print("Initial Available Customers:")
    print(available_customers)
    print()

    for sales_id in sales_ids:
        # Debugging: Print the current available customers before assignment
        print(f"Assigning customers for {sales_id}... Current available customers:")
        print(available_customers)
        print()

        num_customers = random.randint(2, min(len(available_customers), 6))
        customers = random.sample(available_customers, k=num_customers)
        assigned_customers[sales_id] = customers
        for customer in customers:
            available_customers.remove(customer)

        # Debugging: Print assigned customers and remaining available customers
        print(f"{sales_id} assigned customers: {customers}")
        print("Remaining available customers:")
        print(available_customers)
        print()

    # Debugging: Final assignment of customers
    print("Final Assigned Customers for Each Sales Representative:")
    for sales_id, customers in assigned_customers.items():
        print(f"{sales_id}: {customers}")
    print()

    # You can proceed with the rest of your data generation logic here...
    return assigned_customers
